In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Nueva sección

In [0]:
df = pd.read_excel('/content/drive/My Drive/Proyecto SaturdaysAI - Prediccion de ludopatia/EstudioPrevalencia2015.xlsx', 1)

In [0]:
### Importación de las librerías necesarias ###
###############################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier

from pylab import rcParams

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
from collections import Counter

from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectKBest

%matplotlib inline

In [0]:
df.describe()

,numero,modalidad,P0R1,P0B,POB1,P0B2,P0C,P0F1,POF2,POF3,POf3_2,POF4,P1P1A,P1P1B,P1P1C,P1P1D,P1P1E,P1P1F,P1P1G,P1P1H,P1P1I,P1P1J,P1P1K,P1P1L,P1P1M,P1P1N,P1P1O,P2P2A,P2P2B,P2P2C,P2P2D,P2P2E,P2P2F,P2P2G,P2P2H,P2P2I,P2P2J,P2P2K,P2P2L,P2P2M,...,P38SP5,P38SP6,P38SP7,P38SP8,P38SP9,P38SP10,P39,P40,P41,P42,P43,P44,P45_1,P45_2,P45_3,P45R1,P45R2,P45R3,P45R4,P45R5,P45R6,P45R7,P46,P47,P48,P50,DB0,DB1,DB2,DB3,DB4,DB5,DB6,DB6B,DB7,DB8,PESO,PESO1,revisadas_personales,revisadas_telefónicas
count,6816.000000,6816.0,6816.000000,6816.000000,6816.000000,6816.000000,6816.000000,6816.000000,5202.000000,4669.000000,513.0,413.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,...,6816.000000,6816.000000,6816.000000,6816.000000,6816.00000,6815.000000,6816.000000,766.000000,6816.000000,800.000000,6816.000000,112.000000,0.0,0.0,0.0,19.0,17.0,24.0,39.0,0.0,14.0,10.000000,6816.000000,413.000000,0.0,413.000000,6816.000000,6816.000000,3326.000000,6816.000000,6816.000000,6816.000000,6816.000000,149.000000,6816.000000,6816.000000,6816.000000,6816.000000,660.0,1307.0
mean,4577.223738,1.0,8.456719,3.571156,5.777142,47.907864,1.516432,1.236796,1.102461,1.911544,1.0,1.421308,2.280854,3.031719,2.469819,1.348904,1.534987,1.782391,1.198385,1.461169,1.230873,1.152057,1.317378,1.261246,1.377547,1.139946,1.233948,1.097463,1.096886,1.078047,1.071703,1.038831,1.104191,1.048827,1.058824,1.069012,1.070742,1.100346,1.193579,1.071895,...,5.915200,2.525381,4.259096,4.766285,5.83216,2.301834,2.490464,1.643603,2.313674,1.606250,2.370745,1.669643,NaN,NaN,NaN,1.0,2.0,3.0,4.0,NaN,100.0,69.200000,8.720511,5.414044,NaN,4.888620,2.964935,2.785358,9.767889,30.568369,39.057072,2.367811,2.843603,5.993289,1.098005,2.488116,1.050457,1.046409,1.0,1.0
std,2076.015852,0.0,4.826821,1.763138,1.626472,17.571003,0.499767,0.425147,0.303282,0.283987,0.0,0.494368,2.884986,3.466758,4.366012,3.917181,3.221017,3.578914,3.870408,4.767759,3.621480,3.347890,2.467673,4.119956,4.153850,2.750614,3.864965,2.386738,1.460837,1.976030,2.370355,1.392578,2.397426,1.939064,1.948481,1.954010,1.957635,1.981438,3.368312,1.956414,...,4.516035,4.334096,5.093613,5.520358,5.72554,6.455216,7.672013,0.479247,6.499160,0.488886,6.142334,0.472456,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,46.372406,4.313207,18.245241,NaN,17.710514,3.990058,6.669132,12.835252,43.814978,46.042025,6.428943,6.887837,1.646030,2.087890,7.864340,0.181920,0.182712,0.0,0.0
min,1006.000000,1.0,1.000000,1.000000,3.000000,18.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,1.0,2.0,3.0,4.0,NaN,100.0,2.000000,1.000000,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,1.000000,0.000000,0.175572,0.131613,1.0,1.0
25%,2777.750000,1.0,5.000000,2.000000,4.000000,34.000000,1.000000,1.000000,1.000000,2.000000,1.0,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,1.000000,1.000000,1.000000,4.00000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,NaN,NaN,NaN,1.0,2.0,3.0,4.0,NaN,100.0,26.000000,8.000000,2.000000,NaN,1.000000,2.000000,1.000000,6.000000,2.000000,4.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.082998,1.082998,1.0,1.

In [0]:
# Declaración de funciones para limpiar el Dataframe

def change_99_100(x):
    if (type(x) == int or type(x) == float) and x >= 99.0:
      return None
    else :
      return x

def change_column_P23(x):
    if (type(x) == int or type(x) == float) and x >= 2.0:
      return 2.0
    else :
      return x

def cleanDataset(df):
  new_df = df.dropna(axis=1, how='all')
  print("Empty dropped values: " + str(df.shape[1]-new_df.shape[1]))
  new_df = new_df.drop(['numero', 'modalidad'],1)
  print("Eliminadas columnas que no aportan valor ", new_df.shape)
  new_df = new_df[new_df['P23'] <= 3]
  print("Eliminadas filas que no han contestado a la variable objetivo ", new_df.shape )
  # new_df = new_df.apply(lambda x: x >= 99 if None else x)
  new_df = new_df.apply(change_99_100)
  print("Eliminados los valores de 99 o 100", new_df.shape)
  new_df['P23'] = new_df['P23'].apply(change_column_P23)
  print("Unificados Valores de P23 2 y 3 en solo 2", new_df.shape)
  return new_df

In [0]:
# Análisis de variables con valores escasos

print(new_df.shape)
sparse_values_df = pd.DataFrame()
for column in new_df.columns:
  if new_df[column].count() < int(0.2*new_df.shape[0]):
    sparse_values_df[column] = new_df[column]

print(sparse_values_df.shape)
sparse_values_df.head()

(5159, 217)
(5159, 78)


,POf3_2,POF4,P3B,P10R3,P10R4,P10R6,P10R7,P10R8,P10R10,P10R11,P10R12,P10R12Otros_B,P12B_1,P14B1,P14B2,P14B3,P14B4,P14B5,P14B6,P14B7,P14B8,P14B9,P14B10,P14B11,P14B12,P14B13,P14B14,P14B15,P14B16,P14B17,P15B,P15B1R1,P15B1R2,P15B1R3,P15B1R5,P15B1R6,P15B1R7,P15B1R8,P15B1R9,P15B1R10,P15B1R11,P15B1R12,P15B1R13,P15B1R14,P15B1R15,P15B1R17,P15B1R18,P16B,P16C,P16D,P19R1,P19R2,P19R3,P19R4,P19R5,P19R6,P19R7,P19R10,P19R12,P19R13,P19R14,P19R16,P19R17,P19R18,P40,P42,P44,P45R1,P45R2,P45R3,P45R4,P45R6,P45R7,P47,P50,DB6B,revisadas_personales,revisadas_telefónicas
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [0]:
new_df.columns

In [0]:
### Variable Objetivo ###
plot= new_df['P23'].value_counts().plot(kind='barh', title="Variable objetivo") 

In [0]:
### Tabla Variable objetivo - Comunidad autónoma ###
pd.crosstab(new_df['P23'], new_df['P0R1'], margins=True)

In [0]:
sns.set()
%config InlineBackend.figure_format='retina'
sns.countplot(x='P0R1', hue='P23', data=new_df);

In [0]:
new_df['P23'].unique()

In [0]:
### Histograma de la edad ###
sns.distplot(new_df['P0B2'])

In [0]:
### Histograma del sexo ###
sns.distplot(new_df['P0C'])

In [0]:
### Histograma Estado de Salud ###
sns.distplot(new_df['P28'])